In [1]:
# Supporting Libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle
from torchvision import ops

# Models
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor

## Functions

In [2]:
def decode_prediction(prediction, 
                      score_threshold = 0.8, 
                      nms_iou_threshold = 0.2):
    """
    Inputs
        prediction: dict
        score_threshold: float
        nms_iou_threshold: float
    Returns
        prediction: tuple
    """
    boxes = prediction["boxes"]
    scores = prediction["scores"]
    labels = prediction["labels"]    
    # Remove any low-score predictions.
    if score_threshold is not None:
        want = scores > score_threshold
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]    
    # Remove any overlapping bounding boxes using NMS.
    if nms_iou_threshold is not None:
        want = ops.nms(boxes = boxes, scores = scores, iou_threshold = nms_iou_threshold)
        boxes = boxes[want]
        scores = scores[want]
        labels = labels[want]    
        return (boxes.numpy(), labels.numpy(), scores.numpy())
    
def get_best_counts(df, preds, thresh):
    best_possible_count = []
    for name in df["File Name"]:
        opt_score = df[df["File Name"] == name]["Score"].values[0]
        b_count = 0
        for pred in preds[name]:
                    boxes, scores, labels = decode_prediction(pred, opt_score, thresh)
                    b_count += len(boxes)
        best_possible_count.append(b_count)
    return best_possible_count

def get_counts(df, preds, pred_scores, thresh):
    count = []
    for i in range(df.shape[0]):
        row = df.iloc[i, :]
        file_name = row["File Name"]
        image_count = 0
        score = pred_scores[i]
        for pred in preds[file_name]:
              boxes, scores, labels = decode_prediction(pred, score, thresh)
              image_count += len(boxes)
        count.append(image_count)
    return count

def get_metrics(actual_count, pred_count):
    ac = np.array(actual_count)
    pc = np.array(pred_count)
    average_error = abs(ac - pc).mean()
    average_percent_error = np.nan_to_num(abs(ac - pc)/ac, nan = pc).mean()
    median_error = np.median(abs(ac-pc))
    median_percent_error = np.median(np.nan_to_num(abs(ac - pc)/ac, nan = pc))
    return average_error, average_percent_error, median_error, median_percent_error

def eval_predictor(df, pred_info, pred_score, thresh, actual):
    pred_counts = get_counts(df, pred_info, pred_score, thresh)
    if actual:
         return get_metrics(df["Actual Count"], pred_counts)
    else:
        return get_metrics(df["Best Possible Counts"], pred_counts)

def get_total_boxes(df, preds, thresh):
    total_box_num = []
    for name in df["File Name"]:
        total_box_count = 0
        for pred in preds[name]:
            boxes, scores, labels = decode_prediction(pred, 0, thresh)
            total_box_count += len(boxes)
        total_box_num.append(total_box_count)
    return total_box_num  

def sci_get_metrics(model, x_train, y_train, x_val, df_training, df_validation, training_preds, val_preds, thresh, actual, baseline):
    model.fit(x_train, y_train)
    preds = model.predict(x_train) + baseline
    train_ae, train_ape, train_me, train_mpe = eval_predictor(df_training, training_preds, preds, thresh, actual)
    preds = model.predict(x_val) + baseline
    val_ae, val_ape, val_me, val_mpe = eval_predictor(df_validation, val_preds, preds, thresh, actual)
    return train_ae, train_ape, val_ae, val_ape, train_me, train_mpe, val_me, val_mpe

def get_default_metrics(val, df_training, training_preds, df_val, val_preds, y_train, thresh, actual):
     train_ae, train_ape, train_me, train_mpe = eval_predictor(df_training, training_preds, [val] * len(y_train), thresh, actual)
     val_ae, val_ape, val_me, val_mpe = eval_predictor(df_val, val_preds, [val] * len(y_train), thresh, actual)
     return train_ae, train_ape, val_ae, val_ape, train_me, train_mpe, val_me, val_mpe

def generate_comparison(models, model_names, df_training, df_validation, training_preds, val_preds, thresh, default_scores, actual, baseline = 0):
    training_aes = []
    training_apes = []
    val_aes = []
    val_apes = []
    training_mes = []
    training_mpes = []
    val_mes = []
    val_mpes = []
    y_train = df_training["Score"] - baseline
    x_train = df_training.drop(columns = ["Score", "File Name", "Best Possible Counts", "Unnamed: 0", "Actual Count"])
    x_val = df_validation.drop(columns = ["Score", "File Name", "Best Possible Counts", "Unnamed: 0", "Actual Count"])
    for model in models:
        train_ae, train_ape, val_ae, val_ape, train_me, train_mpe, val_me, val_mpe = sci_get_metrics(model, x_train, y_train, x_val, df_training, df_validation, training_preds, val_preds, thresh, actual, baseline)
        training_aes.append(train_ae)
        training_apes.append(train_ape)
        val_aes.append(val_ae)
        val_apes.append(val_ape)
        training_mes.append(train_me)
        training_mpes.append(train_mpe)
        val_mes.append(val_me)
        val_mpes.append(val_mpe)
    for val in default_scores:
        train_ae, train_ape, val_ae, val_ape, train_me, train_mpe, val_me, val_mpe = get_default_metrics(val, df_training, training_preds, df_validation, val_preds, y_train, thresh, actual)
        training_aes.append(train_ae)
        training_apes.append(train_ape)
        val_aes.append(val_ae)
        val_apes.append(val_ape)
        training_mes.append(train_me)
        training_mpes.append(train_mpe)
        val_mes.append(val_me)
        val_mpes.append(val_mpe)
    
    d1 = {"Model Name": model_names, "Training Average Error": training_aes, "Training Average Percent Error":training_apes, "Validation Average Error":val_aes, "Validation Average Percent Error":val_apes}
    d2 =  {"Model Name": model_names, "Training Median Error": training_mes, "Training Median Percent Error":training_mpes, "Validation Median Error":val_mes, "Validation Median Percent Error":val_mpes}
    return pd.DataFrame(d1).set_index("Model Name"), pd.DataFrame(d2).set_index("Model Name")

def get_least_score(grid_df):
    df = grid_df.groupby("Score")["Count Difference"].sum().reset_index()
    return df[df["Count Difference"] == df["Count Difference"].min()]["Score"].values[0]

def distribution_analysis(grid_df):
    df = grid_df.groupby(["File Name", "Score"]).min()
    min_scores = df[df["Count Difference"] == df["Count Difference"].min()].reset_index()[["File Name", "Score"]]
    score_counts = min_scores.groupby("Score").count().reset_index()
    scores = []
    for i in range(score_counts.shape[0]):
        row = score_counts.iloc[i, :]
        for x in range(int(row["File Name"])):
            scores.append(row["Score"])
    scores = np.array(scores)
    bins = np.arange(0, 1, .05)
    plt.hist(scores, bins)
    plt.xticks(np.arange(0, 1, .1))
    plt.xlabel("Optimum Score")
    plt.ylabel("Occurrence")
    plt.show()
    print("Mean: {}".format(scores.mean()))
    print("Median: {}".format(np.median(scores)))
    print("Standard Deviation: {}".format(np.std(scores)))

def error_distribution(model, model_name, df, preds, thresh, df_val = None, percent = True, actual = False):
    if actual:
        if df_val is None:
            ac = df["Actual Count"]
        else:
            ac = df_val["Actual Count"]
    else:
        if df_val is None:
            ac = df["Best Possible Counts"]
        else:
            ac = df_val["Best Possible Counts"]
    y = df["Score"]
    x = df.drop(columns = ["Score", "File Name", "Best Possible Counts", "Unnamed: 0", "Actual Count"])
    model.fit(x, y)
    if df_val is None:
        pred_scores = model.predict(x)
        pred_counts = get_counts(df, preds, pred_scores, thresh)

    else:
        x_val = df_val.drop(columns = ["Score", "File Name", "Best Possible Counts", "Unnamed: 0", "Actual Count"])
        pred_scores = model.predict(x_val)
        pred_counts = get_counts(df_val, preds, pred_scores, thresh)
    pc = np.array(pred_counts)
    if percent:
        error = np.nan_to_num(abs(ac - pc)/ac, nan = pc)
    else:
        error = abs(ac - pc)
    plt.hist(error)
    if percent:
        plt.xlabel("Absolute Count Percent Error")
    else:
        plt.xlabel("Absolute Count Error")
    plt.title("Error Occurence for {}".format(model_name))
    plt.ylabel("Occurrence")
    plt.show()
    print("Mean Error: {}".format(error.mean()))
    print("Median Error: {}".format(np.median(error)))

### RCNN V2

In [3]:
# Default Values

# Thesh of .2


# EPS is 300 instead of 150 and includes all data

# df_training = pd.read_csv("../Data/training_total_0.2_300_V2")
# df_validation = pd.read_csv("../Data/validation_total_0.2_300_V2")
# thresh = .2


# thresh of .05

#EPS is 300 and includes all data

# df_training = pd.read_csv("../Data/training_total_0.05_300_V2")
# df_validation = pd.read_csv("../Data/validation_total_0.05_300_V2")
# thresh = .05


df_training = pd.read_csv("../Data/training_total_0.2_300_new_model_40")
df_validation = pd.read_csv("../Data/validation_mean_0.2_300_new_model_40")
thresh = .2

In [4]:
# df_training["Total Boxes"] = get_total_boxes(df_training, training_preds, thresh)
# df_validation["Total Boxes"] = get_total_boxes(df_validation, val_preds, thresh)

In [5]:
training_pred_path = r"C:\Users\kaanan\Desktop\RCNN\MetaData\training_preds_new_model_30"
val_pred_path = r"C:\Users\kaanan\Desktop\RCNN\MetaData\validation_preds_new_model_30"
grid_search_training_path = r"C:\Users\kaanan\Desktop\RCNN\Data\grid_seach_training_total_new_model_30.csv"
grid_search_val_path = r"C:\Users\kaanan\Desktop\RCNN\Data\grid_seach_validation_total_new_model_30.csv"

with open(training_pred_path, "rb") as fp:
    training_preds = pickle.load(fp)

with open(val_pred_path, "rb") as fp:
    val_preds = pickle.load(fp)

grid_train = pd.read_csv(grid_search_training_path)
grid_val = pd.read_csv(grid_search_val_path)

### Score Model Evaluation

#### Compared to Best Possible Counts

In [6]:
# No RGB values
# df_training = df_training[["File Name", "Box Num", "Score", "Actual Count", "Cluster Num", "Biggest Cluster", "Smallest Cluster", 'Unnamed: 0']]
# df_validation = df_validation[["File Name", "Box Num", "Score", "Actual Count", "Cluster Num", "Biggest Cluster", "Smallest Cluster", 'Unnamed: 0']]

In [7]:
df_training["Best Possible Counts"] = get_best_counts(df_training, training_preds, thresh)
df_validation["Best Possible Counts"] = get_best_counts(df_validation, val_preds, thresh)

In [8]:
best_train_score = get_least_score(grid_train)
best_val_score = get_least_score(grid_val)
model_names = ["KNN Regressor", "Linear Regression", "Random Forest Regression", "Best Score for Training ({})".format(best_train_score), "Best Score for Validation ({})".format(best_val_score)]
models = [KNeighborsRegressor(), LinearRegression(), RandomForestRegressor(random_state=0)]
df_mean, df_median = generate_comparison(models, model_names, df_training, df_validation, training_preds, val_preds, thresh, [best_train_score, best_val_score], False)
df_mean

C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:57: RuntimeWarning: invalid value encountered in divide
  average_percent_error = np.nan_to_num(abs(ac - pc)/ac, nan = pc).mean()
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:59: RuntimeWarning: invalid value encountered in divide
  median_percent_error = np.median(np.nan_to_num(abs(ac - pc)/ac, nan = pc))
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:57: RuntimeWarning: invalid value encountered in divide
  average_percent_error = np.nan_to_num(abs(ac - pc)/ac, nan = pc).mean()
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:59: RuntimeWarning: invalid value encountered in divide
  median_percent_error = np.median(np.nan_to_num(abs(ac - pc)/ac, nan = pc))
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:57: RuntimeWarning: invalid value encountered in divide
  average_percent_error = np.nan_to_num(abs(ac - pc)/ac, nan = pc).mean()
C:\Users\kaanan\AppData\

,Training Average Error,Training Average Percent Error,Validation Average Error,Validation Average Percent Error
Model Name,,,,
KNN Regressor,5.149457,0.148808,27.6875,0.372660
Linear Regression,6.258152,0.201467,54.1250,0.452074
Random Forest Regression,3.904891,0.185252,25.2500,0.285740
Best Score for Training (0.95),8.173913,0.263313,43.8125,0.233789
Best Score for Validation (0.9),7.505435,0.226311,30.3125,0.182247


#### Compared to Actual Counts

In [10]:
df_mean, df_median = generate_comparison(models, model_names, df_training, df_validation, training_preds, val_preds, thresh, [best_train_score, best_val_score], True)
df_mean

C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:57: RuntimeWarning: invalid value encountered in divide
  average_percent_error = np.nan_to_num(abs(ac - pc)/ac, nan = pc).mean()
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:59: RuntimeWarning: invalid value encountered in divide
  median_percent_error = np.median(np.nan_to_num(abs(ac - pc)/ac, nan = pc))
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:57: RuntimeWarning: invalid value encountered in divide
  average_percent_error = np.nan_to_num(abs(ac - pc)/ac, nan = pc).mean()
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:59: RuntimeWarning: invalid value encountered in divide
  median_percent_error = np.median(np.nan_to_num(abs(ac - pc)/ac, nan = pc))
C:\Users\kaanan\AppData\Local\Temp\ipykernel_4348\3316255726.py:57: RuntimeWarning: invalid value encountered in divide
  average_percent_error = np.nan_to_num(abs(ac - pc)/ac, nan = pc).mean()
C:\Users\kaanan\AppData\

,Training Average Error,Training Average Percent Error,Validation Average Error,Validation Average Percent Error
Model Name,,,,
KNN Regressor,8.578804,0.426973,40.3750,0.600887
Linear Regression,7.991848,0.365408,44.8125,0.546558
Random Forest Regression,6.910326,0.359556,37.0625,0.486748
Best Score for Training (0.95),7.771739,0.231875,25.7500,0.265007
Best Score for Validation (0.9),7.864130,0.299939,19.2500,0.268273
